Okay so this is where I will be using pytorch to do some sentiment analysis using my gpu
I will be using the imdb data set, load it up
Tokenize it.
Make embeddings out of it, (matter of fact it's gonna happen as I train)
I will train a tansformer model to learn sentiment distibution from the data
If it is not overfitting then I will be able to stack layers until it does
If it starts to overfit i will randomly drop some data in the hidden layer


In [ ]:
import torch
from torchtext.data.utils import get_tokenizer  # Updated location
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
help(build_vocab_from_iterator)

In [ ]:
import os

imdb_dir = os.path.join(os.getcwd(), 'aclImdb')

train_dir = os.path.join(imdb_dir, 'test')

train_dir

In [4]:
labels = []
texts = []
for label in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label)
    for fname in os.listdir(dir_name):  # Change 'file' to 'fname'
        # Check if it is a text file
        if fname.endswith('.txt'):
            with open(os.path.join(dir_name, fname), 'r', encoding='utf-8') as f:
                texts.append(f.read())
                if label == 'neg':
                    labels.append(0)
                else:
                    labels.append(1)


In [ ]:
help(build_vocab_from_iterator)

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(texts), specials = ["<unk>", "<pad>"], max_tokens=20000)

vocab.set_default_index(vocab["<unk>"])  # Set <unk> as the default for out-of-vocab words



In [7]:
from torch.nn.utils.rnn import pad_sequence

def numericalize(texts, max_len):
    sequences = []
    for text in texts:
        tokens = tokenizer(text)
        numericalized_tokens = [vocab[token] for token in tokens]

        if len(numericalized_tokens) > max_len:
            numericalized_tokens = numericalized_tokens[:max_len]
        else:
            numericalized_tokens.extend([vocab["<pad>"]] * (max_len - len(numericalized_tokens)))

        sequences.append(torch.tensor(numericalized_tokens))
    
    return torch.stack(sequences)


In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# Convert texts and labels to tensors
label_tensor = torch.tensor(labels)
numericalized_texts = numericalize(texts, max_len=300)

# Create dataset and dataloaders
dataset = TensorDataset(numericalized_texts, label_tensor)
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Optionally, print out the sizes of the splits
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 12070
Validation dataset size: 3448
Test dataset size: 1725


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
print(torch.__version__)
print(torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())  # Should print True if CUDA is available

2.3.0
11.8
CUDA available: True


In [ ]:
device
# check if the GPU is available

device(type='cuda')

In [12]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Model definition
class SentimentModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, nhead, num_layers, hidden_dim, output_dim):
        super(SentimentModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim, 
            nhead=nhead, 
            dim_feedforward=hidden_dim, 
            activation='relu',
            dropout = 0.5,
            batch_first=True  # Ensure [batch_size, seq_len, emb_dim] order
        )
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc_dropout = nn.Dropout(0.5)  # 50% dropout rate
        self.fc = nn.Linear(emb_dim, output_dim)
        
        
    def forward(self, x, src_mask=None):
        embedded = self.embedding(x)  # shale of the embedding [batch_size, seq_len, emb_dim]
       
        src_key_padding_mask = (x == vocab["<pad>"])  # Use pad_idx to detect padding tokens

        

        # Pass through the transformer with the padding mask
        transformer_output = self.transformer(
            src=embedded, 
            src_key_padding_mask=src_key_padding_mask, 
        )

        transformer_output = self.fc_dropout(transformer_output)

        # Use the output of the last token or mean pooling
        
        output = transformer_output.mean(dim=1)
        
        final_output = self.fc(output)  # [batch_size, output_dim]

        return final_output

In [14]:
model = SentimentModel(vocab_size=20000, emb_dim=256, nhead=8, num_layers=3, hidden_dim=512, output_dim=2)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [15]:
def train_model(model, train_loader, val_loader, num_epochs=10):
    train_losses, val_losses, val_accuracies = [], [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch in train_loader:
            texts, labels = batch
            texts, labels = texts.to(device), labels.to(device)  # Move to GPU

            optimizer.zero_grad()
            outputs = model(texts)
            # print('mobel outputs', outputs)
            loss = criterion(outputs, labels)
            # print('this is the training loss', loss)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in val_loader:
                texts, labels = batch
                texts, labels = texts.to(device), labels.to(device)  # Move to GPU

                outputs = model(texts)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct / total
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {avg_val_loss:.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}")

    return train_losses, val_losses, val_accuracies

In [ ]:
train_losses, val_losses, val_accuracies = train_model(model, train_loader, val_loader, num_epochs=10)
torch.save(model.state_dict(), os.getcwd())

In [17]:
from sklearn.metrics import classification_report

def test_model(model, test_loader):
    test_accuracy = 0
    test_loss = 0
    correct = 0  # Initialize correct predictions count
    total = 0    # Initialize total samples count
    all_pred = []
    all_labels = []
    
    # Set the model to evaluation mode
    model.eval()
    
    # Disable gradient calculation during testing to save memory and computation
    with torch.no_grad():  # Corrected: added parentheses to use torch.no_grad as a context manager
        for test_batch in test_loader:
            texts, labels = test_batch
            texts, labels = texts.to(device), labels.to(device)  # Move to GPU if needed

            # Forward pass: get predictions
            predictions = model(texts)
            loss = criterion(predictions, labels)
            test_loss += loss.item()  # Fixed: remove arguments from item()

            # predictions is a tensor of shape (batch_size, num_classes)
            # For example:
            # [
            #     [2, 8],    -> 80% probability for class 1
            #     [6, 4],    -> 60% probability for class 0
            #     [9, 1],    -> 90% probability for class 0
            #     [3, 7],    -> 70% probability for class 1
            # ]
            # We want to find the index of the max value in each row, which is the predicted label
            _, pred_label = torch.max(predictions, 1)  # Get predicted class indices

            all_pred.extend(pred_label.cpu().numpy())  # Corrected: changed 'extends' to 'extend'
            all_labels.extend(labels.cpu().numpy())    # Corrected: changed 'extends' to 'extend'
            
            correct += (pred_label == labels).sum().item()  # Count correct predictions
            total += labels.size(0)  # Count total samples
            
    # Calculate test accuracy
    test_accuracy = correct / total
    avg_test_loss = test_loss / len(test_loader)  # Average test loss
    
    # Print or return results
    report = classification_report(all_labels, all_pred, target_names=['Class 0', 'Class 1'])  # Use your actual class names
    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print(report)


In [18]:
test_model(model, test_loader)

Test Loss: 0.4478, Test Accuracy: 0.9043
              precision    recall  f1-score   support

     Class 0       0.92      0.94      0.93      1226
     Class 1       0.85      0.81      0.83       499

    accuracy                           0.90      1725
   macro avg       0.89      0.88      0.88      1725
weighted avg       0.90      0.90      0.90      1725



In [ ]:
prompt ="This product is great! I love it. It works perfectly and exceeded my expectations. Highly recommend it to everyone!"
tokenized_prompt = numericalize([prompt], max_len=300) 

# Move the tokenized prompt to the same device as the model
print(tokenized_prompt.shape)

tokenized_prompt = tokenized_prompt.to(device)
print(tokenized_prompt)

torch.Size([1, 300])
tensor([[127,   0,  10, 811,   4,  42,  10, 127, 406,   4,  90,  77,  25,  62,
           0, 361,  39,   7,  14,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   

In [26]:
# Set model to evaluation mode
model.eval()

# Inference without gradient calculation
with torch.no_grad():
    output = model(tokenized_prompt)
    probabilities = torch.sigmoid(output)  # Corrected line
    _, label = torch.max(probabilities, 1)
    print(label.item())
    if label.item() == 1:
        print ('positive')
    else:
        print('negative')
    

0
negative


In [ ]:
# Plot the loss and accuracy
epochs = range(1, len(train_losses) + 1)

# Plot training and validation loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, val_accuracies, label='Validation Accuracy', color='g')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [28]:
torch.save(model.state_dict(), 'trained_model.pth')